In [1]:
#The necessary libraries (if you don't have them,
#use pip install (module) or conda install (module))
import pandas as pd
import urllib.request
import os.path
from zipfile import ZipFile
from simpledbf import Dbf5

In [2]:
#Import the data files
#Create a dictionary of the urls and the corresponding zipnames
data_dict={}
data_dict['chem'] = {'url': "https://www.waterboards.ca.gov/drinking_water/certlic/drinkingwater/documents/edtlibrary/chemical.zip",
                    'filename':"chem_import.zip",'filetype':'zip'}
data_dict['storet'] = {'url': "https://www.waterboards.ca.gov/drinking_water/certlic/drinkingwater/documents/edtlibrary/storet.zip",
                      'filename':"storet_import.zip",'filetype':'zip'}
data_dict['geo'] = {'url':"https://data.ca.gov/sites/default/files/Public%20Potable%20Water%20Systems%20FINAL%2006-22-2018_0.csv",
                   'filename':'geo_import.csv','filetype':'csv'}

for k in data_dict.keys():
    dat_temp=data_dict[k]
    if (os.path.exists(dat_temp['filename'])==False):
        print('Retrieving file and saving as {:}'.format(dat_temp['filename']))
        urllib.request.urlretrieve(dat_temp['url'],dat_temp['filename'])


In [7]:
#Unzip the files and store the contents
data_store = {}
for k in data_dict.keys():
        dat_temp=data_dict[k]
        if dat_temp['filetype']=='zip':
            with ZipFile(dat_temp['filename'], 'r') as zipObj:
                fname=zipObj.namelist()[0]
                if (os.path.exists(fname)==False):
                    print("Extracting file {:} from {:}".format(fname,dat_temp['filename']))
                    zipObj.extractall()
                #Open the database file
                print("Opening dbf {:}".format(fname))
                dbf = Dbf5(fname)
                print("Storing data")
                data_store[k] = dbf.to_dataframe()
                print("transformed to data frame")
        else:
            data_store[k] = pd.read_csv(dat_temp['filename'])
            

Opening dbf chemical.dbf
Storing data
transformed to data frame
Extracting file storet.dbf from storet_import.zip
Opening dbf storet.dbf
Storing data
transformed to data frame


In [56]:
#Bit of data cleaning
data_store['chem']['PRIM_STA_C'] = data_store['chem']['PRIM_STA_C'].str[:7]
data_store['geo']=data_store['geo'].rename(index=str,columns={'Water System No ': 'PRIM_STA_C'})
data_store['geo']['PRIM_STA_C'] = data_store['PRIM_STA_C'].str[2:]



In [63]:
#One big data file
chem_all = pd.merge(data_store['chem'],data_store['storet'],on='STORE_NUM',how='left')
chem_all = pd.merge(chem_all,data_store['geo'],on='PRIM_STA_C',how='left')

In [67]:
#Writing this to CSV 
chem_all.to_csv('chem_all.csv',index=False)

In [ ]:
#What are the available 